In [1]:
%pylab inline
from __future__ import print_function, division
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import scipy.stats
from MTD import *
import brewer2mpl
from scipy.stats.mstats import zscore as zscore
import os

Populating the interactive namespace from numpy and matplotlib


In [4]:
#read data
os.chdir('/home/despoB/kaihwang/bin/TTD/Models')
%run dataframe_ts.py
TS_df.head()

,FFA,PPA,VC,Subject,Condition,Dataset,Run,site
0,0.599642,0.258330,0.431444,601,FH,FIR,1,FEF
1,0.284079,0.041826,0.387960,601,FH,FIR,1,FEF
2,0.362462,-0.116999,0.531450,601,FH,FIR,1,FEF
3,0.330204,-0.110393,0.232863,601,FH,FIR,1,FEF
4,-0.047260,-0.097048,0.175755,601,FH,FIR,1,FEF


In [44]:
Subjects = [601, 602, 603, 605]
Conditions = ['FH', 'Fp', 'HF', 'Hp']
sites = ['FEF', 'S1', 'MFG']
#MotorMapping = [1, 2]
corr_df = pd.DataFrame(columns=['Subj', 'site', 'Condition', 'Run','Window', 'MTD-FFA-VC', 'MTD-PPA-VC', 'corr-FFA-VC', 'corr-PPA-VC'], dtype=float)
#'Run', 'FFA-M','FFA-NM', 'PPA-M', 'PPA-NM',
Runs = [1, 2]
#motor mapping 1, RH-Face LH-Scene
#motor mapping 2, RH-Scene, LH-Face

i=1
corr_df =  pd.DataFrame()
for s, sub in enumerate(Subjects):
    for site in sites:
        for c, cond in enumerate(Conditions):
            for run in Runs:
                PPA_ts = np.array(TS_df[(TS_df['Subject'] == sub)  & (TS_df['site'] == site) & (TS_df['Condition'] == cond) & (TS_df['Run'] == run)]['PPA'])
                FFA_ts = np.array(TS_df[(TS_df['Subject'] == sub) & (TS_df['site'] == site) & (TS_df['Condition'] == cond) & (TS_df['Run'] == run)]['FFA'])
                VC_ts = np.array(TS_df[(TS_df['Subject'] == sub) & (TS_df['site'] == site) & (TS_df['Condition'] == cond) & (TS_df['Run'] == run)]['VC'])

                tmpdf = pd.DataFrame()
                MTD_PPA_VC = coupling(np.array([PPA_ts, VC_ts]).T, 10)[1][:,0,1]
                MTD_FFA_VC = coupling(np.array([FFA_ts, VC_ts]).T, 10)[1][:,0,1]
                tmpdf['MTD-PPA-VC'] = MTD_PPA_VC[~np.isnan(MTD_PPA_VC)] 
                tmpdf['MTD-FFA-VC'] = MTD_FFA_VC[~np.isnan(MTD_FFA_VC)] 
                tmpdf['site'] = site
                tmpdf['Condition'] = cond
                tmpdf['Subj'] = sub
                tmpdf['Run'] = run
                tmpdf['Window'] = w
                corr_df = corr_df.append(tmpdf,  ignore_index=True)
                


In [45]:
corr_df.head(20)

,MTD-PPA-VC,MTD-FFA-VC,site,Condition,Subj,Run,Window
0,0.015173,0.091297,FEF,FH,601,1,10
1,0.021806,0.124077,FEF,FH,601,1,10
2,0.059858,0.026825,FEF,FH,601,1,10
3,0.075178,0.014090,FEF,FH,601,1,10
4,0.050877,0.106179,FEF,FH,601,1,10
5,0.058708,0.105879,FEF,FH,601,1,10
6,0.074531,0.180531,FEF,FH,601,1,10
7,-0.178146,0.048165,FEF,FH,601,1,10
8,-0.105618,0.052781,FEF,FH,601,1,10
9,-0.055021,-0.002074,FEF,FH,601,1,10


In [47]:
mdf = corr_df.groupby(['Subj','site','Condition']).mean().reset_index()
sdf = corr_df.groupby(['Subj','site','Condition']).std().reset_index()

In [90]:
Subjects = [601, 602, 603, 605]
sites = ['FEF', 'MFG']

mtd_df =  pd.DataFrame()
for s, sub in enumerate(Subjects):
    for site in sites:
        tmpdf = pd.DataFrame()
        mtds = corr_df[(corr_df['site']==site) & (corr_df['Subj']==sub)& (corr_df['Condition']=='HF')]['MTD-PPA-VC'].values
        m = mdf[(mdf['site']=='S1') & (mdf['Subj']==sub)& (mdf['Condition']=='HF')]['MTD-PPA-VC'].values
        sd = sdf[(sdf['site']=='S1') & (sdf['Subj']==sub)& (sdf['Condition']=='HF')]['MTD-PPA-VC'].values
        
        tmpdf['MTD-Change-Zscore'] = (mtds -m) / sd
        tmpdf['Site'] = site
        tmpdf['Subj'] = sub
        mtd_df = mtd_df.append(tmpdf,  ignore_index=True)


In [92]:
tmpdf = pd.DataFrame()
mtds = corr_df[(corr_df['site']==site) & (corr_df['Subj']==sub)& (corr_df['Condition']=='Fp')]['MTD-FFA-VC'].values
m = mdf[(mdf['site']=='S1') & (mdf['Subj']==sub)& (mdf['Condition']=='Fp')]['MTD-FFA-VC'].values
sd = sdf[(sdf['site']=='S1') & (sdf['Subj']==sub)& (sdf['Condition']=='Fp')]['MTD-FFA-VC'].values
        
tmpdf['MTD-Change-Zscore'] = (mtds -m) / sd
tmpdf['Site'] = 'FEF'
tmpdf['Subj'] = 604
mtd_df = mtd_df.append(tmpdf,  ignore_index=True)

In [93]:
mtd_df.to_csv('mtds.csv')